In [2]:
!pip install streamlit pyodbc langchain langchain_openai pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.2/346.2 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.8 MB/s eta 0:00:00


In [1]:
!sudo apt-get update
!sudo apt-get install curl
!curl https://packages.microsoft.com/keys/microsoft.asc | sudo apt-key add -
!curl https://packages.microsoft.com/config/ubuntu/22.04/prod.list | sudo tee /etc/apt/sources.list.d/mssql-release.list
!sudo apt-get update


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,383 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,688 kB]
Get:13 http://security.ubuntu.com/ubun

In [4]:
!sudo apt-get install msodbcsql17

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  odbcinst unixodbc
The following NEW packages will be installed:
  msodbcsql17 odbcinst unixodbc
0 upgraded, 3 newly installed, 0 to remove and 47 not upgraded.
Need to get 783 kB of archives.
After this operation, 164 kB of additional disk space will be used.
Get:1 https://packages.microsoft.com/ubuntu/22.04/prod jammy/main amd64 msodbcsql17 amd64 17.10.6.1-1 [746 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 odbcinst amd64 2.3.9-5ubuntu0.1 [9,930 B]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 unixodbc amd64 2.3.9-5ubuntu0.1 [26.7 kB]
Fetched 783 kB in 0s (2,763 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: 

In [6]:
!odbcinst -q -d

[ODBC Driver 17 for SQL Server]


In [ ]:
%%writefile app.py
import streamlit as st
import pyodbc
from langchain_openai import ChatOpenAI
import re

st.set_page_config(page_title="EV Fleet Chatbot", page_icon="🚗")
st.header("EV Fleet Chatbot")
st.write("Ask me about your fleet data and operations!")

DB_CONFIG = {
    "SERVER": "S.Name/IP",
    "DATABASE": "DB.Name",
    "USER": "U.Name",
    "PASSWORD": "P/W",
    "DRIVER": "{ODBC Driver 17 for SQL Server}"
}

def configure_llm():
    return ChatOpenAI(temperature=0, openai_api_key="Enter key here")

def connect_to_db():
    try:
        conn = pyodbc.connect(
            f"DRIVER={DB_CONFIG['DRIVER']};"
            f"SERVER={DB_CONFIG['SERVER']};"
            f"DATABASE={DB_CONFIG['DATABASE']};"
            f"UID={DB_CONFIG['USER']};"
            f"PWD={DB_CONFIG['PASSWORD']}"
        )
        return conn
    except Exception as e:
        st.error(f"Database connection failed: {e}")
        return None

def extract_vehicle_id(query):
    match = re.search(r'\b\d{4,}\b', query)
    return match.group() if match else None

def fetch_latest_vehicle_data(vehicle_id, conn, query_type):
    cursor = conn.cursor()
    query = """
        SELECT TOP 1 Speed, Latitude, Longitude, Odometer, MessgeTypeName
        FROM tmpmsginhst
        WHERE VehicleId = ?
    """
    cursor.execute(query, (vehicle_id,))
    row = cursor.fetchone()

    if row:
        data = {
            "speed": row.Speed,
            "location": f"({row.Latitude}, {row.Longitude})",
            "odometer": row.Odometer,
            "last_harsh_braking": None
        }

        if query_type == "harsh braking":
            braking_query = """
                SELECT TOP 1 Timestamp
                FROM tmpmsginhst
                WHERE VehicleId = ? AND MessgeTypeName = 'HarshBraking'
            """
            cursor.execute(braking_query, (vehicle_id,))
            braking_row = cursor.fetchone()
            if braking_row:
                data["last_harsh_braking"] = braking_row.Timestamp

        return data
    return None

def generate_response(user_query, vehicle_data):
    llm = configure_llm()
    prompt = f"""
    Generate a natural language response for a fleet tracking chatbot.
    User asked: '{user_query}'
    Data available: {vehicle_data}
    Respond in a concise and friendly manner.
    """
    response = llm.invoke(prompt)
    return response

def chatbot_interface():
    conn = connect_to_db()
    if not conn:
        st.error("Database not connected. Please check the connection.")
        return

    user_query = st.text_input("Enter your query:")
    if user_query:
        vehicle_id = extract_vehicle_id(user_query)

        if vehicle_id:
            query_type = "general"
            if "speed" in user_query.lower():
                query_type = "speed"
            elif "location" in user_query.lower() or "where is" in user_query.lower():
                query_type = "location"
            elif "odometer" in user_query.lower():
                query_type = "odometer"
            elif "harsh braking" in user_query.lower():
                query_type = "harsh braking"

            vehicle_data = fetch_latest_vehicle_data(vehicle_id, conn, query_type)
            if vehicle_data:
                response = generate_response(user_query, vehicle_data)
                st.write(response.content if hasattr(response, 'content') else "Unable to generate a response.")
            else:
                st.write(f"No recent data found for vehicle {vehicle_id}.")
        else:
            st.write("Please provide a valid Vehicle ID in your query.")

if __name__ == "__main__":
    chatbot_interface()


Overwriting app.py


In [ ]:
!pip install pyngrok
!ngrok ngrok_token

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!streamlit run app.py &> logs.txt &

In [ ]:
from pyngrok import ngrok
ngrok.kill()
public_url = ngrok.connect(8501)
print("Public URL:", public_url)


Public URL: NgrokTunnel: "https://83d4-35-202-162-154.ngrok-free.app" -> "http://localhost:8501"
